# 1. Inicializando projeto

In [16]:
from dotenv import dotenv_values

config = dotenv_values("../../.env")

pinecone_api_ley = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]
index_pinecone = config["INDEX_NAME"]

openai_api_key = config["OPENAI_API_KEY"]

## 1.1 Inicializar cliente Pinecone (configurar conexão)

In [17]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_ley)
index = pc.Index(index_pinecone)

# Verificando se está tudo ok
print(pc.list_indexes())

[{
    "name": "livreto-base-evidencia",
    "metric": "cosine",
    "host": "livreto-base-evidencia-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}]


# 2. Extrair texto do PDF

In [18]:
from pathlib import Path

path = Path("LangChain/RAG/Data/Livreto_BaseEvidencia_2019.11.21.pdf")
print("Existe?", path.exists())
print("Caminho absoluto:", path.resolve())

Existe? False
Caminho absoluto: C:\Users\irani\Desktop\Personal code\LLM\Using LLM\LangChain\RAG\LangChain\RAG\Data\Livreto_BaseEvidencia_2019.11.21.pdf


In [19]:
import pdfplumber
import json

path = "Data/Livreto_BaseEvidencia_2019.11.21.pdf"
pages=[]
with pdfplumber.open(path) as pdf:
    for i , p in enumerate(pdf.pages, start=1):
        text = p.extract_text() or ""
        pages.append({"page": i, "text": text})

with open("data/pdf_pages_livreto.json","w",encoding="utf-8") as f:
    json.dump(pages,f,ensure_ascii=False,indent=2)

# Limpeza e normalização de texto

In [20]:
import re

def limpar_texto(texto: str) -> str:
    if not texto:
        return ""

    # Remove caracteres estranhos comuns de OCR
    texto = re.sub(r'[^\wÀ-ÿ\s.,;:!?()\-"“”]', ' ', texto)

    # Remove letras soltas em linhas (ex: O\nI\nR\n)
    texto = re.sub(r'(?:\b[A-ZÀ-Ý]\b\s*){3,}', ' ', texto)

    # Normaliza quebras de linha
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'\n', ' ', texto)

    # Normaliza espaços
    texto = re.sub(r'\s{2,}', ' ', texto)

    return texto.strip()

def limpar_json_paginas(paginas: list) -> list:
    json_limpo = []

    for pagina in paginas:
        texto_limpo = limpar_texto(pagina.get("text", ""))

        # ignora páginas vazias após limpeza
        if texto_limpo:
            json_limpo.append({
                "page": pagina["page"],
                "text": texto_limpo
            })

    return json_limpo


In [21]:
import json

with open("data/pdf_pages_livreto.json", encoding="utf-8") as f:
    paginas = json.load(f)

json_limpo = limpar_json_paginas(paginas)

with open("data/pdf_pages_livreto_clean.json", "w", encoding="utf-8") as f:
    json.dump(json_limpo, f, ensure_ascii=False, indent=2)


# Dividir texto em pedaços menores (Chunks)

In [30]:
# Carregando dados
from langchain_core.documents import Document
with open('data/pdf_pages_livreto_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

paginas_texto = {}
for item in data:
    p = item['page']
    t = item['text']
    if p not in paginas_texto:
        paginas_texto[p] = t
    else:
        # Une fragmentos da mesma página com uma quebra de linha
        paginas_texto[p] += "\n" + t

# Convertendo para objetos Document do LangChain (Lista de objetos consolidados)
documents = [
    Document(page_content=texto, metadata={"page": pagina}) 
    for pagina, texto in paginas_texto.items()
]


# Estratégia de Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, # Suficiente para captar o tópico inteiro de uma página
    chunk_overlap=0,
    separators=["I DEFINIÇÃO", "II MOBILIZAÇÃO", "III DETERMINANTES", 
                "IV SOLUÇÃO", "V JUSTIFICATIVA", "VI APRIMORAMENTO", 
                "VII CERTIFICAÇÃO", "\n\n", ". "]
)


chunks = text_splitter.split_documents(documents)

In [36]:
print("Conferindo 'a cara' dos chunks!")
print(chunks[13].page_content) 

Conferindo 'a cara' dos chunks!
V JUSTIFICATIVA Toda solução demanda recursos. Para ser justificada, uma solução precisa que o valor do que é capaz de alcançar (seus benefícios) supere o valor.dos recursos que necessita (seus custos). Embora a justificativa definitiva de uma solução só possa ocorrer após a sua impantação, a decisão por adotá-la precisa ser baseada em avaliações ex-ante de sua relação custo-efetividade e custo-benefício. AVITACIFITSUJ “If whatever it is you re explaining has some measure ... you ll be much better able to discriminate among competing hypotheses. What is vague ... is open to many explanations” Carl Sagan


# Embeddings + Ingestão no Pinecone

In [37]:
from langchain_openai import OpenAIEmbeddings

# Inicializando o modelo de embeddings da OpenAI
# O modelo 'text-embedding-3-small' é o mais custo-benefício atualmente
embeddings_model = OpenAIEmbeddings(
    api_key=openai_api_key, 
    model="text-embedding-3-small"
)

In [38]:
from langchain_pinecone import PineconeVectorStore

# Enviando os chunks (sua lista de Document) para o Pinecone
# O LangChain vai automaticamente gerar os embeddings e salvar no Index
vector_store = PineconeVectorStore.from_documents(
    documents=chunks, # Usando a variável 'chunks'
    embedding=embeddings_model,
    index_name=index_pinecone,
    pinecone_api_key=pinecone_api_ley
)

print(f"Sucesso! {len(chunks)} documentos foram indexados no Pinecone.")

Sucesso! 22 documentos foram indexados no Pinecone.


# Teste

In [39]:
# Texto de um novo documento que você deseja classificar
query = """
O orçamento de 2026 prevê cortes severos nas bolsas da CAPES e CNPq, 
com reduções de até 18% e 25%, respectivamente. A Lei Orçamentária Anual (LOA) 
de 2026 aponta uma queda de R$ 359,3 milhões para a CAPES. Esses recursos 
foram redirecionados para emendas parlamentares em ano eleitoral, 
gerando grande preocupação na comunidade acadêmica quanto à formação de pesquisadores."""

# Busca os documentos mais parecidos
docs = vector_store.similarity_search(query, k=1)

if docs:
    print("\n--- Classificação Sugerida ---")
    print(f"Conteúdo do Guia: {docs[0].page_content[:200]}...")
    print(f"Página Original: {docs[0].metadata['page']}")


--- Classificação Sugerida ---
Conteúdo do Guia: V JUSTIFICATIVA Valoração do custo Impacto (eficácia) Recursos são sempre escassos. Assim, justificar, com base em evidência a adoção de uma ação requer tanto estimativas da magnitude do seu impacto q...
Página Original: 16


# Consulta e recuperação de dados

In [ ]:
# Instancia o retriever (configurado para trazer o chunk mais relevante)
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

# O retriever é um motor de busca.
# 1. Ele transforma a sua query_usuario em um vetor.
# 2. Ele vai ao Pinecone e procura o chunk que tem a maior "similaridade de cosseno" (o texto mais parecido semanticamente).
# 3. k=1: Você configurou para ele trazer apenas o melhor resultado. Se o usuário fala de "orçamento", o retriever traz a página de custos.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos o modelo (LLM)
llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key, temperature=0)

# 2. Criamos o template com 'espaços vazios' destinados a variáveis dinâmicas
template = """
Você é um especialista em Gestão Pública baseada em Evidências.
Sua tarefa é classificar um novo texto em uma das etapas do ciclo de políticas públicas, 
utilizando ESTRITAMENTE o contexto fornecido abaixo.

CONTEXTO DO GUIA INSTITUCIONAL:
{context}

TEXTO PARA CLASSIFICAÇÃO:
{question}

RESPOSTA ESPERADA:
1. Nome da Etapa (Ex: I DEFINIÇÃO, II MOBILIZAÇÃO, etc)
2. Justificativa curta (máximo 2 linhas) explicando por que se encaixa nessa etapa.
3. Página de referência do guia original.
"""

# 3. O ChatPromptTemplate lê o objeto 'tamplate' e substitui as variáveis dinâmicas em seus respectivos lugares
prompt = ChatPromptTemplate.from_template(template)

# 3. Pipeline completa 
chain = (
    {"context": retriever, "question": RunnablePassthrough()} # O RunnablePassthrough() age como um "placeholder" que diz à chain: "Espere até o momento do .invoke() para pegar a query
    | prompt
    | llm
    | StrOutputParser() # Limpeza: Remove as rebarbas da API e entrega apenas o produto final (Texto).
)


query_usuario = """
O orçamento de 2026 prevê cortes severos nas bolsas da CAPES e CNPq, 
com reduções de até 18% e 25%, respectivamente. A Lei Orçamentária Anual (LOA) 
de 2026 aponta uma queda de R$ 359,3 milhões para a CAPES. Esses recursos 
foram redirecionados para emendas parlamentares em ano eleitoral, 
gerando grande preocupação na comunidade acadêmica quanto à formação de pesquisadores.
"""

resposta = chain.invoke(query_usuario)
print(resposta)

1. Nome da Etapa: V JUSTIFICATIVA
2. Justificativa curta: O texto aborda a valoração de custos e o impacto de cortes orçamentários, elementos centrais na justificativa de políticas.
3. Página de referência do guia original: 16.
